In [1]:
import pandas as pd
import numpy as np 

import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv('johns-hopkins-covid-19-daily-dashboard-cases-by-states.csv')

In [3]:
df.head()

,province_state,country_region,last_update,lat,long,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate
0,Alabama,US,2020-04-29 12:32:29,32.3182,-86.9023,6750,242,NaN,6508.0,1.0,143.957097,75827.0,911.0,3.585185,84000001,USA,1617.160710,13.496296
1,Alaska,US,2020-04-29 12:32:29,61.3707,-152.4044,351,9,228.0,342.0,2.0,58.720299,17089.0,NaN,2.564103,84000002,USA,2858.892278,NaN
2,American Samoa,US,2020-04-29 12:32:29,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN
3,Arizona,US,2020-04-29 12:32:29,33.7298,-111.4312,6955,275,1450.0,6680.0,4.0,95.552554,67438.0,1095.0,3.953990,84000004,USA,926.509438,15.744069
4,Arkansas,US,2020-04-29 12:32:29,34.9697,-92.3731,3127,57,1146.0,3070.0,5.0,120.778964,40671.0,NaN,1.822833,84000005,USA,1570.899029,NaN


In [4]:
df['ds'] = df.last_update

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 19 columns):
province_state          141 non-null object
country_region          141 non-null object
last_update             141 non-null object
lat                     135 non-null float64
long                    135 non-null float64
confirmed               141 non-null int64
deaths                  141 non-null int64
recovered               119 non-null float64
active                  140 non-null float64
fips                    58 non-null float64
incident_rate           135 non-null float64
people_tested           56 non-null float64
people_hospitalized     30 non-null float64
mortality_rate          137 non-null float64
uid                     141 non-null int64
iso3                    141 non-null object
testing_rate            56 non-null float64
hospitalization_rate    30 non-null float64
ds                      141 non-null object
dtypes: float64(11), int64(3), object(5)
memory usage: 2

In [6]:
df['ds'] = pd.to_datetime(df['ds'])

In [7]:
df = df.drop(columns=['last_update'])

In [8]:
df.head()

,province_state,country_region,lat,long,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate,ds
0,Alabama,US,32.3182,-86.9023,6750,242,NaN,6508.0,1.0,143.957097,75827.0,911.0,3.585185,84000001,USA,1617.160710,13.496296,2020-04-29 12:32:29
1,Alaska,US,61.3707,-152.4044,351,9,228.0,342.0,2.0,58.720299,17089.0,NaN,2.564103,84000002,USA,2858.892278,NaN,2020-04-29 12:32:29
2,American Samoa,US,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN,2020-04-29 12:32:29
3,Arizona,US,33.7298,-111.4312,6955,275,1450.0,6680.0,4.0,95.552554,67438.0,1095.0,3.953990,84000004,USA,926.509438,15.744069,2020-04-29 12:32:29
4,Arkansas,US,34.9697,-92.3731,3127,57,1146.0,3070.0,5.0,120.778964,40671.0,NaN,1.822833,84000005,USA,1570.899029,NaN,2020-04-29 12:32:29


In [9]:
(df.isna().sum()) / len(df) * 100

province_state           0.000000
country_region           0.000000
lat                      4.255319
long                     4.255319
confirmed                0.000000
deaths                   0.000000
recovered               15.602837
active                   0.709220
fips                    58.865248
incident_rate            4.255319
people_tested           60.283688
people_hospitalized     78.723404
mortality_rate           2.836879
uid                      0.000000
iso3                     0.000000
testing_rate            60.283688
hospitalization_rate    78.723404
ds                       0.000000
dtype: float64

## missing a lot of data...

dropping people tested, fips, people_hospitalized, testing rate, and hospitalization_rate

In [10]:
df = df.drop(columns=['fips', 'people_tested', 'people_hospitalized', 'testing_rate', 'hospitalization_rate'])

In [11]:
df.isna().sum()

province_state     0
country_region     0
lat                6
long               6
confirmed          0
deaths             0
recovered         22
active             1
incident_rate      6
mortality_rate     4
uid                0
iso3               0
ds                 0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 13 columns):
province_state    141 non-null object
country_region    141 non-null object
lat               135 non-null float64
long              135 non-null float64
confirmed         141 non-null int64
deaths            141 non-null int64
recovered         119 non-null float64
active            140 non-null float64
incident_rate     135 non-null float64
mortality_rate    137 non-null float64
uid               141 non-null int64
iso3              141 non-null object
ds                141 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(3), object(3)
memory usage: 14.4+ KB


In [13]:
mask = df.mortality_rate.isna()

In [14]:
df[mask]

,province_state,country_region,lat,long,confirmed,deaths,recovered,active,incident_rate,mortality_rate,uid,iso3,ds
2,American Samoa,US,-14.271,-170.132,0,0,NaN,NaN,0.0,NaN,16,ASM,2020-04-29 12:32:29
115,Recovered,Canada,NaN,NaN,0,0,19231.0,-19231.0,NaN,NaN,12415,CAN,2020-04-29 12:32:29
116,Recovered,US,NaN,NaN,0,0,115936.0,-115936.0,NaN,NaN,84070001,USA,2020-04-29 12:32:29
139,Diamond Princess,Canada,NaN,NaN,0,1,0.0,-1.0,NaN,NaN,12413,CAN,2020-04-29 12:32:29


In [15]:
df = df.drop([2, 115, 116, 139])

In [18]:
df.isna().sum()

province_state     0
country_region     0
lat                3
long               3
confirmed          0
deaths             0
recovered         21
active             0
incident_rate      3
mortality_rate     0
uid                0
iso3               0
ds                 0
dtype: int64

In [21]:
mask = df.recovered.isna()

df[mask]

,province_state,country_region,lat,long,confirmed,deaths,recovered,active,incident_rate,mortality_rate,uid,iso3,ds
0,Alabama,US,32.3182,-86.9023,6750,242,NaN,6508.0,143.957097,3.585185,84000001,USA,2020-04-29 12:32:29
5,California,US,36.1162,-119.6816,46164,1864,NaN,44300.0,117.739415,4.037778,84000006,USA,2020-04-29 12:32:29
7,Connecticut,US,41.5978,-72.7554,26312,2087,NaN,24225.0,738.005103,7.931742,84000009,USA,2020-04-29 12:32:29
11,Florida,US,27.7663,-81.6868,32848,1171,NaN,31677.0,154.705456,3.564905,84000012,USA,2020-04-29 12:32:29
12,Georgia,US,33.0406,-83.6431,24923,1036,NaN,23887.0,245.803669,4.156803,84000013,USA,2020-04-29 12:32:29
17,Illinois,US,40.3495,-88.9861,48102,2125,NaN,45977.0,409.879195,4.417696,84000017,USA,2020-04-29 12:32:29
18,Indiana,US,39.8494,-86.2583,16588,901,NaN,15687.0,253.484340,5.431637,84000018,USA,2020-04-29 12:32:29
20,Kansas,US,38.5266,-96.7265,3652,127,NaN,3525.0,149.764936,3.477547,84000020,USA,2020-04-29 12:32:29
25,Massachusetts,US,42.2302,-71.5301,58302,3153,NaN,55149.0,849.416327,5.408048,84000025,USA,2020-04-29 12:32:29
28,Mississippi,US,32.7416,-89.6787,6342,239,NaN,6103.0,219.051770,3.768527,84000028,USA,2020-04-29 12:32:29


In [28]:
df.recovered.fillna(0, inplace = True)

In [29]:
df.isna().sum()

province_state    0
country_region    0
lat               3
long              3
confirmed         0
deaths            0
recovered         0
active            0
incident_rate     3
mortality_rate    0
uid               0
iso3              0
ds                0
dtype: int64

In [32]:
mask = df.incident_rate.isna()

df[mask]

,province_state,country_region,lat,long,confirmed,deaths,recovered,active,incident_rate,mortality_rate,uid,iso3,ds
9,Diamond Princess,US,NaN,NaN,49,0,0.0,49.0,NaN,0.000000,84088888,USA,2020-04-29 12:32:29
13,Grand Princess,US,NaN,NaN,103,3,0.0,100.0,NaN,2.912621,84099999,USA,2020-04-29 12:32:29
140,Grand Princess,Canada,NaN,NaN,13,0,0.0,13.0,NaN,0.000000,12414,CAN,2020-04-29 12:32:29


In [33]:
df.incident_rate.fillna(0, inplace=True)

In [34]:
df.isna().sum()

province_state    0
country_region    0
lat               3
long              3
confirmed         0
deaths            0
recovered         0
active            0
incident_rate     0
mortality_rate    0
uid               0
iso3              0
ds                0
dtype: int64

In [35]:
df.fillna(0, inplace=True)

In [36]:
df = df.sort_values('ds').set_index('ds')